In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras.models import Sequential # type: ignore
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout # type: ignore
from keras.optimizers import Adam
import cv2 as cv
from PIL import Image

In [3]:
# Load the data
split_df = pd.read_csv('split_df.csv', encoding='utf-8-sig')


In [4]:
split_df

,Table 1,Spectacles,Age,Gender,Name,id,Eye,VA,CDR,Diagnosis,IOP
0,زرق أيمن تم معالجته جراحيا,0.0,64,0,شاهين ميرو,1.0,0,0.6,0.5,0,14.0
1,زرق أيمن تم معالجته جراحيا,0.0,64,0,شاهين ميرو,1.0,1,0.7,0.9,1,23.0
2,زرق متقدم,1.0,62,0,أيمن سمارة,2.0,0,NaN,NaN,1,25.0
3,زرق متقدم,1.0,62,0,أيمن سمارة,2.0,1,9.0,0.8,1,40.0
4,لا يوجد زرق,0.0,48,1,رندة ناصر,3.0,0,1.0,0.5,0,15.0
...,...,...,...,...,...,...,...,...,...,...,...
85,زرق أيسر تم معالجته جراحيا,1.0,55,1,زريفة المحمد,43.0,1,NaN,NaN,0,NaN
86,مريض زرقي ثانوي بعد استخراج ساد لا يوجد اصابة ...,0.0,71,0,هيثم قزاز,44.0,0,0.3,0.5,1,15.0
87,مريض زرقي ثانوي بعد استخراج ساد لا يوجد اصابة ...,0.0,71,0,هيثم قزاز,44.0,1,0.6,0.7,1,16.0
88,لا يوجد زرق,1.0,65,0,حمود غشام,45.0,0,0.5,0.3,0,15.0


In [5]:
split_df['Age'].value_counts()

Age
64    10
65     6
25     4
67     4
73     4
62     4
55     4
58     4
70     4
50     2
59     2
29     2
17     2
48     2
37     2
52     2
32     2
45     2
60     2
44     2
69     2
43     2
12     2
66     2
24     2
53     2
56     2
39     2
68     2
61     2
76     2
71     2
Name: count, dtype: int64

In [143]:
split_df = split_df.drop(['Table 1', 'Gender', 'id', 'Age', 'Spectacles'], axis=1)

In [144]:
split_df['Image'].isnull().sum()

12

In [145]:
# split_df['Image']
split_df[split_df['Image'].isnull()]

,Name,Eye,VA,CDR,Diagnosis,IOP,Image
2,أيمن سمارة,0,NaN,NaN,1,25.0,None
9,نهلة أبو قورة,1,1.0,NaN,1,11.0,None
50,حصة الحمد,0,0.5,0.3,0,NaN,None
52,ايمان الزعبي,0,1.0,0.3,0,18.0,None
53,ايمان الزعبي,1,1.0,0.3,0,19.0,None
62,أسرار صنوبر,0,1.0,NaN,0,18.0,None
63,أسرار صنوبر,1,9.0,NaN,0,18.0,None
82,عنود الحميد,0,0.4,0.3,0,18.0,None
83,عنود الحميد,1,0.5,0.3,0,17.0,None
85,زريفة المحمد,1,NaN,NaN,0,NaN,None


In [146]:
split_df = split_df.drop(['Name','Eye'], axis=1)

In [147]:
split_df.dropna(inplace=True)

In [148]:
split_df

,VA,CDR,Diagnosis,IOP,Image
0,0.6,0.5,0,14.0,Data/SHAHEN_MERO Optic nerve.png_ left.png
1,0.7,0.9,1,23.0,Data/SHAHEN_MERO Optic nerve.png_ right.png
3,9.0,0.8,1,40.0,Data/AYMAN_SAMARA_20240121_124719_Disc_3D_R_SI...
4,1.0,0.5,0,15.0,Data/RAMA_NASR_ALL Optic nerve.png_ left.png
5,1.0,0.4,0,14.0,Data/RANDA_NASER_2 Optic nerve.png_ right.png
...,...,...,...,...,...
80,0.7,0.8,0,13.0,Data/MOUHAMAD_HELAL_ Optic nerve.png_ left.png
81,9.0,0.8,1,16.0,Data/MOUHAMAD_HELAL_ Optic nerve.png_ left.png
84,0.2,0.8,1,10.0,Data/ZAREFA_MOUHAMAD_20231227_103707_Disc_3D_L...
86,0.3,0.5,1,15.0,Data/HAETHAM_QAZAZ Optic nerve.png_ left.png


In [149]:
orb = cv.ORB_create()

def extract_orb_features(image_path):
    image = cv.imread(image_path)
    gray = cv.cvtColor(image, cv.COLOR_BGR2GRAY)
    keypoints, descriptors = orb.detectAndCompute(gray, None)
    return descriptors.flatten() if descriptors is not None else np.zeros(500)

In [150]:
split_df['ORB_features'] = split_df['Image'].apply(extract_orb_features)

In [151]:
split_df.count() 

VA              64
CDR             64
Diagnosis       64
IOP             64
Image           64
ORB_features    64
dtype: int64

In [152]:
# Load and preprocess images for each patient
image_paths =  split_df['Image'].tolist()


In [153]:
image_paths

['Data/SHAHEN_MERO Optic nerve.png_ left.png',
 'Data/SHAHEN_MERO Optic nerve.png_ right.png',
 'Data/AYMAN_SAMARA_20240121_124719_Disc_3D_R_SINGLE_6mm_512x112.png',
 'Data/RAMA_NASR_ALL Optic nerve.png_ left.png',
 'Data/RANDA_NASER_2 Optic nerve.png_ right.png',
 'Data/MOUSTAFA_MALA_ALI Optic nerve.png_ left.png',
 'Data/MOUSTAFA_MALA_ALI Optic nerve.png_ right.png',
 'Data/LATEFA_IBRAHEM Optic nerve.png_ left.png',
 'Data/LATEFA_IBRAHEM Optic nerve.png_ right.png',
 'Data/FATEMA_FARHAN Optic nerve.png_ left.png',
 'Data/FATEMA_FARHAN Optic nerve.png_ right.png',
 'Data/SOBHI_WAZAN Optic nerve.png_ left.png',
 'Data/SOBHI_WAZAN Optic nerve.png_ right.png',
 'Data/HIBA_LOLO Optic nerve.png_ left.png',
 'Data/HIBA_LOLO Optic nerve.png_ right.png',
 'Data/KHADEGA_KABEH Optic nerve.png_ left.png',
 'Data/KHADEGA_KABEH Optic nerve.png_ right.png',
 'Data/ABD_ALRAZAQ_BOEDANI Optic nerve.png_ left.png',
 'Data/ABD_ALRAZAQ_BOEDANI Optic nerve.png_ right.png',
 'Data/5919_HALEMA_SHEKHO Optic 

In [154]:
# images = [np.array(Image.open(path).convert('RGB').resize((200, 200))) for path in image_paths]  # Load and resize images
# X_images = np.array(images) / 255.0  # Normalize pixel values to range [0, 1]

In [155]:
# Drop columns not used for prediction
y_data = split_df['Diagnosis']
X_data = split_df.drop(['Image','Diagnosis'], axis=1)

In [156]:
# # image_paths[0]
# img = cv.imread('Data/SHAHEN_MERO Optic nerve.png_ left.png')
# cv.imshow('Image', img)

# cv.waitKey(0)
# cv.destroyAllWindows()

In [121]:
# X_images = []
# # cv.imshow('',X_images)
# # cv.waitKey(0)
# # cv.destroyAllWindows()
# for i in range(len(image_paths)):
#     img = cv.imread(image_paths[i])
#     flattened_image = img.flatten()
#     X_images.append(flattened_image)

In [122]:
# X_images = pd.DataFrame(X_images,columns='images')
# X_images = pd.DataFrame({'Image': X_images})


In [123]:
# X_images

,Image
0,"[187, 184, 176, 200, 197, 189, 200, 197, 189, ..."
1,"[101, 90, 59, 72, 55, 9, 72, 55, 9, 72, 55, 9,..."
2,"[68, 52, 9, 69, 53, 9, 70, 54, 9, 71, 54, 9, 7..."
3,"[190, 187, 176, 204, 200, 189, 204, 200, 189, ..."
4,"[107, 95, 60, 84, 64, 11, 89, 68, 12, 92, 70, ..."
...,...
59,"[187, 184, 176, 201, 197, 189, 201, 197, 189, ..."
60,"[187, 184, 176, 201, 197, 189, 201, 197, 189, ..."
61,"[62, 47, 8, 58, 44, 7, 60, 45, 7, 64, 48, 7, 6..."
62,"[188, 186, 176, 203, 200, 189, 204, 200, 189, ..."


In [124]:
# X_images.isnull().sum()

Image    0
dtype: int64

In [157]:

X_data

,VA,CDR,IOP,ORB_features
0,0.6,0.5,14.0,"[13, 254, 157, 23, 145, 206, 39, 184, 242, 223..."
1,0.7,0.9,23.0,"[218, 169, 144, 178, 155, 153, 16, 206, 171, 5..."
3,9.0,0.8,40.0,"[198, 98, 254, 209, 238, 107, 251, 200, 255, 1..."
4,1.0,0.5,15.0,"[69, 55, 237, 5, 226, 106, 45, 24, 106, 250, 6..."
5,1.0,0.4,14.0,"[16, 12, 3, 191, 20, 199, 235, 98, 54, 72, 82,..."
...,...,...,...,...
80,0.7,0.8,13.0,"[190, 223, 119, 210, 187, 245, 158, 244, 153, ..."
81,9.0,0.8,16.0,"[190, 223, 119, 210, 187, 245, 158, 244, 153, ..."
84,0.2,0.8,10.0,"[156, 116, 122, 229, 35, 91, 255, 248, 53, 134..."
86,0.3,0.5,15.0,"[229, 190, 159, 62, 168, 173, 144, 175, 158, 2..."


In [158]:
X_data.isnull().sum()

VA              0
CDR             0
IOP             0
ORB_features    0
dtype: int64

In [129]:
# X_train = np.concatenate((X_data, X_images), axis=1)
# X_data['images'] = X_images['Image']
# for i in range(len(X_images)):
#     X_data.at[i, 'images'] = X_images.at[i, 'Image']


In [159]:
X_data.count()

VA              64
CDR             64
IOP             64
ORB_features    64
dtype: int64

In [160]:
# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, test_size=0.1, random_state=44, shuffle=True)


In [133]:
# Define CNN model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(200, 200, 3)),
    MaxPooling2D((2, 2)),
    Dropout(rate=0.2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Dropout(0.2),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(512, activation='relu'),
    Dense(1, activation='sigmoid') 
])

In [134]:
# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [135]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_4 (Conv2D)           (None, 198, 198, 32)      896       
                                                                 
 max_pooling2d_4 (MaxPoolin  (None, 99, 99, 32)        0         
 g2D)                                                            
                                                                 
 dropout_2 (Dropout)         (None, 99, 99, 32)        0         
                                                                 
 conv2d_5 (Conv2D)           (None, 97, 97, 64)        18496     
                                                                 
 max_pooling2d_5 (MaxPoolin  (None, 48, 48, 64)        0         
 g2D)                                                            
                                                                 
 conv2d_6 (Conv2D)           (None, 46, 46, 128)      

In [136]:
# Train the model
history = model.fit(X_train, y_train, epochs=20, batch_size=32, validation_data=(X_test, y_test))


ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type float).

In [ ]:
test_loss, test_acc = model.evaluate(X_images_test, y_test)
print(f"Test Accuracy: {test_acc}")

1/1 [==============================] - 0s 111ms/step - loss: 0.9338 - accuracy: 0.5714
Test Accuracy: 0.5714285969734192


In [ ]:
# Evaluate the model
train_score = model.evaluate(X_images_train, y_train)
test_score = model.evaluate(X_images_test, y_test)
print('CNN Train Score:', train_score)
print('CNN Test Score:', test_score)


1/1 [==============================] - 0s 119ms/step - loss: 0.9338 - accuracy: 0.5714
CNN Train Score: [0.34277939796447754, 0.8245614171028137]
CNN Test Score: [0.9337977170944214, 0.5714285969734192]


In [ ]:
# Make predictions on the test set
predictions = model.predict(X_images_test)


1/1 [==============================] - 0s 284ms/step


In [ ]:
# # Generate classification report and confusion matrix
# print('Classification Report:')
# print(classification_report(y_test, predictions))

# print('Confusion Matrix:')
# print(confusion_matrix(y_test, predictions))


In [ ]:
from keras.backend import clear_session
clear_session()